## Aula 1 - Solução dos exercícios
Leandro Carísio Fernandes

<br>

Enunciado do exercício da semana:

1. Usar o BM25 implementado pelo pyserini para buscar queries no TREC-DL 2020 ([Documentação de referência](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md))
2. Implementar um buscador booleano/bag-of-words.
3. Implementar um buscador com TF-IDF
4. Avaliar implementações 1, 2, e 3 no TREC-DL 2020 e calcular o nDCG@10

Nos itens 2 e 3: (i) Fazer uma implementação que suporta buscar eficientemente milhões de documentos e (ii) Não se pode usar bibliotecas como sklearn, que já implementam o BoW e TF-IDF.

<br>



### I. Conceitos

Inicialmente fiquei muito perdido com toda essa história. O que é anserini? pyserini? MSMARCO? Passage? Documents? TREC-DL 2020? Como isso tudo se conecta?

Então vale a pena começar vendo o que é cada uma dessas coisas:

1. Passage/documents

  Perguntei ao ChatGPT "what is the difference between document and passage (context: information retrieval)?" e a resposta foi:

  <br>

  "In the context of information retrieval, a "document" refers to a complete unit of text, such as a full article, a book, or a web page. *A document contains multiple passages or sections of text that may or may not be relevant to a user's query*.

  On the other hand, *a "passage" refers to a shorter section of text*, usually a few sentences long, that is directly relevant to a user's query. Passages are often extracted from larger documents and presented to the user as the most relevant part of the document for their query. (...)"

  <br>

  "Passage" parece ser apenas trechos de texto mesmo. Isso pode ser visto também no [survey](https://arxiv.org/pdf/2010.06467.pdf). Por exemplo (capítulo 2):

  <br>

  "*Approaches to segmenting documents into passages* for ranking purposes and integrating evidence from multiple document granularities — commonly referred to as passage retrieval — was an active area of research in the 1990s (...)."

<br>

2. MS MARCO

  [De acordo com o site](https://microsoft.github.io/msmarco/):

  <br>

  "Starting with a paper released at [NIPS 2016](https://arxiv.org/pdf/1611.09268.pdf), MS MARCO is a collection of datasets focused on deep learning in search.

  The first dataset was a question answering dataset featuring 100,000 real Bing questions and a human generated answer. Since then we released a 1,000,000 question dataset, a natural langauge generation dataset, a passage ranking dataset, keyphrase extraction dataset, crawling dataset, and a conversational search.

  (...)

  "Microsoft Machine Reading Comprehension (MS MARCO) is a collection of large scale datasets for deep learning related to Search. In MS MARCO, all questions are sampled from real anonymized user queries. The context passages, from which answers in the dataset are derived, are extracted from real web documents using the most advanced version of the Bing search engine. The answers to the queries are human generated if they could summarize the answer."

  <br>

  Em suma, é uma coleção de bases de dados usadas para teste. Há a coleção msmarco-passage (passage) e a coleção msmarco-doc (documents)

<br>

3. Anserini, pyserini e TREC

  Perguntei ao ChatGPT "what is anserini?" e a resposta foi:

  <br>

  "Anserini is an open-source information retrieval toolkit developed jointly by the University of Waterloo and the University of Massachusetts Amherst. It provides a set of tools and algorithms for performing various tasks related to information retrieval, such as indexing, searching, and ranking of documents.

  Anserini was developed specifically for the TREC (Text REtrieval Conference) competitions, which are annual conferences focused on information retrieval research. It was designed to be fast, efficient, and scalable, and to provide state-of-the-art retrieval models and evaluation tools.

  Anserini supports a wide range of retrieval models, including classic models such as BM25 and language models like Dirichlet and Jelinek-Mercer smoothing. It also includes more recent neural models, such as the Conv-KNRM and the BERT-IR models, which have achieved state-of-the-art performance on various retrieval tasks.

  In addition to supporting a variety of retrieval models, Anserini also provides tools for indexing large document collections, evaluating retrieval results using standard measures such as mean average precision (MAP) and normalized discounted cumulative gain (NDCG), and processing queries in different formats. (...)"

  <br>

  Perguntei também "how is pyserine related to the trec conference?":

  <br>

  "Pyserini is a Python interface to the Anserini information retrieval toolkit, which was developed by the University of Waterloo and the University of Massachusetts Amherst for the TREC (Text REtrieval Conference) competitions. TREC is an annual conference series focused on information retrieval research, where participants compete to develop the most effective algorithms for various tasks, such as document retrieval, passage retrieval, and question answering.

  Anserini was developed specifically for the TREC 2018 Common Core Track, which aimed to provide a common testbed for research in document retrieval using a standard set of topics and relevance judgments. Since then, Anserini has been used for many other TREC tracks and has become a popular toolkit for information retrieval research.

  Pyserini provides a convenient Python wrapper around the Anserini toolkit, allowing researchers to use Anserini's powerful retrieval models and evaluation tools in a Python environment. This makes it easier to experiment with different retrieval models and compare their performance on various datasets, including those used in TREC competitions. Therefore, Pyserini is a valuable tool for researchers who are interested in participating in the TREC conference or conducting information retrieval research more broadly."

  <br>

  Considerando essas respostas e uma consulta ao [site](https://trec.nist.gov/): Há uma conferência chamada TREC (Text REtrieval Conference) co-patrocinada pelo National Institute of Standards and Technology (NIST) e pelo U.S. Department of Defense. Essa conferência é anual e o objetivo é apoiar a pesquisa dentro da comunidade de recuperação de informação, fornecendo a infraestrutura necessária para avaliação em larga escala de metodologias de recuperação de texto.

  Um workshop TREC consiste em um conjunto de trilhas (tracks), áreas de foco nas quais tarefas de recuperação específicas são definidas. [TREC-DL 2020](https://microsoft.github.io/msmarco/TREC-Deep-Learning-2020) é nome da trilha deep learning do ano 2020.

<br>

4. Conectando tudo:

  A trilha TREC-DL 2020 tem duas tarefas: ranqueamento de trechos de texto (passage) e ranqueamento de documentos. Cada uma dessas tarefas usam "a large human-generated set of training labels" disponibilizado dentro da base MS MARCO (no caso da primeira, são 200 queries). E o Pyserini pode ser usado para acessar essa base e usar alguns modelos/métricas.

### II. Instalando o pyserini e testando:

Inicialmente, vamos fazer a instalação do pyserini seguindo esses tutoriais:

https://github.com/castorini/pyserini/blob/master/docs/installation.md

https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md

In [ ]:
%%time

# Se True, essa variável configura para gerar o índice da coleção MS MARCO usando o comando pyserini.index.lucene e salva o índice no Drive
# Se False, considera que o índice já está salvo no Drive e lê ele direto de lá
gerar_indice_pyserini = False

# Se True, essa variável configura para gerar o índice que foi implementado neste notebook para a coleção MS MARCO e salva no Drive
# Se False, considera que o índice já está salvo no Drive e apenas lê direto de lá e monta manualmente o objeto aqui
gerar_indice_implementado = False

# Já aproveita e monta logo o google drive pra ele solicitar a permissão de uma vez
from google.colab import drive
drive.mount('/content/drive')

!pip install pyserini
!pip install torch
!pip install faiss-cpu

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CPU times: user 174 ms, sys: 40.2 ms, total: 214 ms
Wall time: 19 s


In [ ]:
# É necessário que seja a versão 11. Checa versão:
!java --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


Segundo o tutorial, "to confirm that bag-of-words retrieval is working correctly, you can run the BM25 baseline on the MS MARCO passage ranking task".

Obs.: o comando abaixo vai baixar e gerar o índice. É demorado (~15min) e só usei ele apenas para seguir o roteiro e ver se a instalação das bibliotecas estava ok. Por isso está comentado.

In [ ]:
#!python -m pyserini.search \
#    --topics msmarco-passage-dev-subset \
#    --index msmarco-v1-passage \
#    --output run.msmarco-passage.txt \
#    --output-format msmarco \
#    --bm25

#!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.msmarco-passage.txt

Até aqui parece ok a instalação padrão.

Agora vamos seguir o tutorial passado como exemplo, BM25 Baseline for MS MARCO Passage Retrieval.

A ideia aqui é baixar o MS MARCO, descompactá-lo e gerar o índice.

O primeiro passo é baixar o arquivo collectionandqueries.tar.gz e descompactá-lo:

In [ ]:
%%time

from pathlib import Path
import hashlib

str_arquivo = './collections/msmarco-passage/collectionandqueries.tar.gz'
md5_esperado = '31644046b18952c1386cd4564ba2ae69'

if not Path(str_arquivo).is_file():
  !mkdir collections/msmarco-passage # type: ignore
  !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage # type: ignore
  !tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage # type: ignore
else:
  print('Arquivo já foi baixado, não precisa fazer o download novamente')

with open(str_arquivo, 'rb') as arquivo:
    data = arquivo.read()
    md5_arquivo = hashlib.md5(data).hexdigest()
    del data
    print('MD5 bate com o esperado' if md5_esperado == md5_arquivo else 'MD5 é diferente do esperado, é necessário fazer o download novamente.')

Arquivo já foi baixado, não precisa fazer o download novamente
MD5 bate com o esperado
CPU times: user 2.07 s, sys: 1.38 s, total: 3.45 s
Wall time: 7.49 s


Os passos seguintes são procedimentos demorados. Por isso a ideia é fazê-los só uma vez para a geração do índice e, depois, guardar o índice já gerado no Google Drive e fazer o download dele diretamente. Por isso, primeiro vou rodar o código aqui para gerar o índice e, depois, comentá-lo. Se necessário (se eu apagar os arquivos do drive por exemplo), depois basta descomentar para gerar novamente.

O segundo passo é converter os arquivos da coleção MS MARCO (extensão tsv) para arquivos jsonl do Pyserini (um objeto json por linha). Isso é feito no tutorial usando o script tools/scripts/msmarco/convert_collection_to_jsonl.py . Entretanto, como esse script não existe, primeiro é necessário fazer o download do arquivo.

In [ ]:
%%time

if gerar_indice_pyserini:
  !wget -O convert_collection_to_jsonl.py https://raw.githubusercontent.com/castorini/anserini-tools/7b84f773225b5973b4533dfa0aa18653409a6146/scripts/msmarco/convert_collection_to_jsonl.py  # type: ignore

  !python convert_collection_to_jsonl.py --collection-path collections/msmarco-passage/collection.tsv --output-folder collections/msmarco-passage/collection_jsonl  # type: ignore

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


Agora indexa a coleção:

In [ ]:
%%time

if gerar_indice_pyserini:
  # Gera o índice
  # Observação: coloquei esse comando em uma linha muito grande pq estava dando um erro de formatação que eu não conseguia tirar...
  !python -m pyserini.index.lucene --collection JsonCollection --input collections/msmarco-passage/collection_jsonl --index indexes/lucene-index-msmarco-passage --generator DefaultLuceneDocumentGenerator --threads 9 --storePositions --storeDocvectors --storeRaw # type: ignore

  # Compacta o índice num arquivo indexes.tar.gz
  !tar -czvf indexes.tar.gz indexes/  # type: ignore

  # Copia o arquivo compactado para a pasta do google drive. Se for o caso, é necessário primeiro criar a pasta IA368-DD_deep_learning_busca
  !mkdir '/content/drive/My Drive/IA368-DD_deep_learning_busca/'  # type: ignore
  !cp indexes.tar.gz '/content/drive/My Drive/IA368-DD_deep_learning_busca/'  # type: ignore
else:
  # O código abaixo considera que o índice já está no google drive. Basta baixar ele e descompactar:
  !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/indexes.tar.gz' './indexes.tar.gz'  # type: ignore

  !tar xvfz indexes.tar.gz  # type: ignore

indexes/
indexes/lucene-index-msmarco-passage/
indexes/lucene-index-msmarco-passage/_0_Lucene90_0.doc
indexes/lucene-index-msmarco-passage/_0_Lucene90_0.dvm
indexes/lucene-index-msmarco-passage/_8.fnm
indexes/lucene-index-msmarco-passage/_0.fnm
indexes/lucene-index-msmarco-passage/_4.tvx
indexes/lucene-index-msmarco-passage/_0_Lucene90_0.pos
indexes/lucene-index-msmarco-passage/_5_Lucene90_0.tip
indexes/lucene-index-msmarco-passage/_2.tvd
indexes/lucene-index-msmarco-passage/_4.fdx
indexes/lucene-index-msmarco-passage/_0.nvd
indexes/lucene-index-msmarco-passage/_3.tvx
indexes/lucene-index-msmarco-passage/_8.fdx
indexes/lucene-index-msmarco-passage/_4_Lucene90_0.dvd
indexes/lucene-index-msmarco-passage/_4.fdt
indexes/lucene-index-msmarco-passage/_6.tvm
indexes/lucene-index-msmarco-passage/_7.nvd
indexes/lucene-index-msmarco-passage/_2.fnm
indexes/lucene-index-msmarco-passage/_2.nvd
indexes/lucene-index-msmarco-passage/_6_Lucene90_0.dvd
indexes/lucene-index-msmarco-passage/_1_Lucene90_0.

### III. Usar o BM25 implementado pelo pyserini para buscar queries no TREC-DL 2020

Podemos testar reproduzindo os resultados de [Ma et al](https://castorini.github.io/pyserini/2cr/msmarco-v1-passage.html). O nDCG@10 deve ser de 0.4796. Vamos usar os comandos descritos no link:

In [ ]:
%%time

!python -m pyserini.search.lucene \
  --threads 16 --batch-size 128 \
  --index indexes/lucene-index-msmarco-passage \
  --topics dl20 \
  --output run.dl20.txt \
  --bm25 --k1 0.9 --b 0.4

!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run.dl20.txt  # type: ignore

2023-03-08 09:50:56.843427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 09:50:58.272931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:50:58.273067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:50:58.273093: W tensorfl

Deu pra reproduzir o resultado do nDCG@10 conforme divulgado (0.4796), até aqui parece que as coisas foram montadas corretamente.

Obs.: Na chamada ao pyserini.search.lucene, ao chamar --topics passamos apenas dl20. Na chamada ao trec_eval, passamos dl20-passage. [No exemplo de onde essa chamada foi tirada](https://castorini.github.io/pyserini/2cr/msmarco-v1-passage.html), se fosse usar o dl19 deveria passar dl19-passage nos dois casos parâmetros...

Obs 2.: O que a chamada acima faz é rodar todas as queries do tópico dl20 e salvar em um arquivo run.dl20.txt. Em seguida é feita outra chamada passando esse arquivo como parâmetro para a avaliar a métrica nDCG@10.

É interessante ver o formato desse arquivo:


In [ ]:
!head run.dl20.txt

3505 Q0 3859340 1 14.401300 Anserini
3505 Q0 4711746 2 14.132300 Anserini
3505 Q0 7207815 3 13.934600 Anserini
3505 Q0 3829534 4 13.207600 Anserini
3505 Q0 8285660 5 12.672300 Anserini
3505 Q0 6834658 6 12.421400 Anserini
3505 Q0 3859344 7 12.351500 Anserini
3505 Q0 6834656 8 12.307600 Anserini
3505 Q0 4409525 9 12.275400 Anserini
3505 Q0 7557329 10 12.215300 Anserini


Segundo o ChatGPT, esse arquivo deve ser interpretado da seguinte forma:

In the Pyserini script you provided, the output file run.dl20.txt will contain the search results for the topics specified in the dl20 file.

Each line in the output file corresponds to a single retrieved document for a particular query (i.e., topic). The format of each line is:

    [topic_id] Q0 [doc_id] [rank] [score] [run_name]

where:

- topic_id is the identifier of the topic/query.
- Q0 is a fixed value indicating that the query was a topic query.
- doc_id is the identifier of the retrieved document.
- rank is the rank of the document in the ranked list of retrieved documents for the topic.
- score is the retrieval score assigned to the document by the retrieval system for the topic.
- run_name is a user-defined identifier for the retrieval run.

For example, a typical line in the output file might look like:

    301 Q0 doc1234 1 10.5678 my_run_name

This line indicates that document doc1234 was retrieved as the top-ranked document for query/topic 301, with a retrieval score of 10.5678, and was assigned the identifier my_run_name for the retrieval run.

The output file can be further processed and analyzed using various evaluation metrics and tools, such as trec_eval or pytrec_eval, to compute performance measures such as precision, recall, MAP, and NDCG.

<br>

Agora vamos tentar reproduzir esse resultado usando código. Esse código usou as seguintes fonte: [1](https://github.com/castorini/pyserini/), [2](https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb#scrollTo=03sPnM3wWBfJ).

O primeiro passo é obter as queries do tópico dl20

In [ ]:
from pyserini.search import get_topics

topics = get_topics('dl20')
print(f'{len(topics)} queries total')

print(topics.keys())
print('\nExemplo de query:')
print(topics[735922])

200 queries total
dict_keys([735922, 23849, 514096, 156498, 258062, 703782, 1071750, 436707, 1117817, 1135268, 67316, 132622, 482726, 99005, 655914, 1065636, 1106928, 169208, 246883, 794223, 1104501, 938400, 586148, 1107440, 1108651, 1130705, 1037496, 1134988, 1119543, 1120588, 1064670, 1043135, 1122767, 1127540, 1109850, 449367, 75198, 144862, 735482, 48792, 1107315, 1116380, 1135283, 88495, 1108473, 91576, 914916, 1110678, 660198, 47210, 877809, 1106979, 792635, 918162, 1108466, 1056416, 1113256, 64647, 1119118, 1134939, 883915, 50122, 730539, 999466, 808400, 1105860, 819983, 330501, 1108450, 701453, 166046, 125659, 1122843, 940547, 1126523, 1121879, 1118370, 174463, 1132532, 302846, 1103791, 555530, 330975, 814183, 121171, 611953, 118440, 850358, 273695, 1112142, 177604, 253749, 997622, 197312, 1135413, 1127233, 655526, 519025, 768208, 1132943, 1134207, 332593, 978031, 256942, 452915, 257119, 1134680, 181626, 809525, 1132950, 1131069, 42752, 390360, 1133485, 1127004, 1128456, 110579

Agora podemos criar um LuceneSearcher carregando o índice que foi gerado e fazer uma pesquisa usando os mesmos parâmetros k1 e b para comparar os resultados (nota: bm25, k1=0.9 e b=0.4 já são default do LuceneSearcher, mas é mais claro deixar explícito do código).

O resultado é um array de hits, cujo conteúdo pode ser acessado através do atributo raw. Por exemplo, para pesquisar pela mesma query acima (what is crimp oil):

In [ ]:
%%time

from pyserini.search.lucene import LuceneSearcher
import json

searcher = LuceneSearcher('indexes/lucene-index-msmarco-passage')
searcher.set_bm25(k1=float(0.9), b=float(0.4))

hits = searcher.search(topics[735922]['title'])

# Prints the first 10 hits
for i in range(0, 10):
    jsondoc = json.loads(hits[i].raw)
    print(f'{i+1:2} {hits[i].score:.5f} {jsondoc["contents"][:80]}...')

# Verifica o formato do objeto hits:
print('\nObjeto hits[0].raw (chegando o formato):')

hits[0].raw

 1 9.60990 Definition of crimped in the Definitions.net dictionary. Meaning of crimped. Wha...
 2 9.39730 Description: There has always been a debate as to what is the best form of crimp...
 3 9.39330 What does crimped mean? Definitions for crimped Here are all the possible meanin...
 4 9.20850 Effect of Crimp Depth on Shotshells. Crimp depth of a finished shotshell reload ...
 5 8.95510 Crimp Oil is a 100% natural blend of essential oils and plant extracts that aids...
 6 8.66260 Directions. 1  Heat your oven to 225 degrees Fahrenheit. 2  On a large sheet of ...
 7 8.56560 1 Heat your oven to 225 degrees Fahrenheit. 2  On a large sheet of aluminum foil...
 8 8.53480 Metolius Crimp Oil. 1  A healing massage oil for climbers hands and muscles. 2  ...
 9 8.49770 crimped. simple past tense and past participle of crimp The sharp bend had crimp...
10 8.47610 Metolius Crimp Oil aids in quick and fast healing of the finger, hand, and wrist...

Objeto hits[0].raw (chegando o formato):
CPU time

'{\n  "id" : "7307871",\n  "contents" : "Definition of crimped in the Definitions.net dictionary. Meaning of crimped. What does crimped mean? Information and translations of crimped in the most comprehensive dictionary definitions resource on the web."\n}'

Agora podemos rodar todas as queries do tópico. O arquivo gerado sobrescreverá o anterior (run.dl20.txt). Como estamos avaliando nDCG@10, vamos pegar apenas os 10 primeiros hits:

In [ ]:
%%time

def run_all_queries(file, topics, searcher):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in topics:
            query = topics[id]['title']
            hits = searcher.search(query, 10) # Gera só os 10 primeiros resultados da query
            for i in range(0, len(hits)):
                _ = runfile.write('{} Q0 {} {} {:.6f} Anserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')

run_all_queries('run.dl20.txt', topics, searcher)

Running 200 queries in total
100 queries completed
200 queries completed
CPU times: user 15 s, sys: 249 ms, total: 15.2 s
Wall time: 6.16 s


Novamente, vamos checar o arquivo gerado e ver todas as queries disponíveis:

In [ ]:
!head -12 run.dl20.txt

print('\nQueries:\n')
for idx in topics:
  print(f'{idx}\t\t{topics[idx]["title"]}')

735922 Q0 7307871 1 9.609900 Anserini
735922 Q0 2766952 2 9.397300 Anserini
735922 Q0 7307863 3 9.393300 Anserini
735922 Q0 8734268 4 9.208500 Anserini
735922 Q0 8626892 5 8.955100 Anserini
735922 Q0 5537112 6 8.662600 Anserini
735922 Q0 5537109 7 8.565600 Anserini
735922 Q0 8626887 8 8.534800 Anserini
735922 Q0 7307868 9 8.497700 Anserini
735922 Q0 8626890 10 8.476100 Anserini
23849 Q0 4348282 1 10.066300 Anserini
23849 Q0 2674124 2 9.865500 Anserini

Queries:

735922		what is crimp oil
23849		are naturalization records public information
514096		the after hours clinic
156498		do google docs auto save
258062		how long does it take to remove wisdom tooth
703782		what is a torn disc
1071750		why is pete rose banned from hall of fame
436707		largest known insects
1117817		what does unauthorized act in writing mean
1135268		antibiotics for what kind of infection
67316		can fever cause miscarriage early pregnancy
132622		definition of attempted arson
482726		projective definition
99005		co

Pra finalizar, vamos calcular o nDCG@10 novamente para confirmar o valor (0.4796):

In [ ]:
%%time

!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run.dl20.txt

2023-03-08 09:51:59.133785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:51:59.133944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:51:59.133974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running

### III. Buscador booleano/bag of words


Pra implementar o buscador é necessário indexar todos os documentos.

Primeiro testei de onde é melhor pegar o conteúdo dos arquivos. Infelizmente não dá pra pegar direto do índice, pois ele só disponibiliza o raw do json convertido a partir do arquivo tsv. Assim, pra acessar o conteúdo de cada documento aqui é necessário primeiro converter pra um dict. Esse processo demorou cerca de 12 minutos. Já acessar diretamente o conteúdo do arquivo collection.tsv leva cerca de 24 segundos apenas.

In [ ]:
#%%time
# Total de documentos indexados no índice
#print(searcher.num_docs)

# Testa se é muito demorado iterar em toda a lista de documentos e converter pra json pra acessar uma propriedade
#for idx in range(searcher.num_docs):
#  doc = searcher.doc(searcher.num_docs-1)
#  contents = json.loads(doc.raw())['contents']
#  if idx % 100000 == 0:
#    print(idx)

# Vamos testar também o tempo pra iterar o arquivo collection.csv e acessar o conteúdo de cada documento:
# !head collections/msmarco-passage/collection.tsv
#with open('collections/msmarco-passage/collection.tsv', encoding='utf-8') as f:
#  for idx, line in enumerate(f):
#    doc_id, contents = line.rstrip().split('\t')
#    if idx % 100000 == 0:
#      print(idx)

Vamos agora criar uma classe para guardar um índice invertido e, em seguida, indexar os quase 9 milhões de documentos da coleção MS MARCO (obs.: como o processo de gerar o índice é demorado, vamos fazer o esquema de gerar uma vez e salvar no Google Drive. Nas próximas pegamos direto de lá).

Observação: O índice está guardando tanto os documentos quanto a quantidade de vezes que o termo ocorre no documento. Por conta de eficiência de memória, a ideia aqui é usar o módulo array. Vou tentar guardar cada inteiro com 4 bytes, mas se não funcionar reduzo o array de ocorrências para 2 bytes. Além disso, para facilitar na hora do TF-IDF, estou guardando um mapa com o total de tokens por documento.

Uma coisa interessante é que o ChatGPT indicou que o tipo i/I tinha 4 bytes, sendo que a documentação do módulo diz que esse tipo de dados são 2 bytes. Ou seja, o ChatGPT indica que é um tipo que dá pra guardar as ids dos documentos (cerca de 8 milhões), entretanto o número máximo é de cerca de 65mil (unsigned)...

In [ ]:
%%time

from pyserini.analysis import Analyzer, get_lucene_analyzer
from collections import Counter
import array

# Definição de uma classe para índice invertido
class IndiceInvertido:

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self, tokenizar = None):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0

    # Se não passou uma função tokenizadora como parâmetro, considera o do Lucene som Porter stemmer:
    if (tokenizar is None):
      tokenizar = Analyzer(get_lucene_analyzer(stemmer='porter')).analyze
    self.tokenizar = tokenizar

  def adiciona_doc(self, id_doc, conteudo_doc=None, tokens=None):
    if (tokens is None):
      tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      # Tenta converter o id do documento para int (ocupa menos memória que uma string)
      # Usa um array de 4bytes para e um de 2bytes para o número de ocorrências
      int_id_doc = int(id_doc)
      self.indice.setdefault(token, {"id_doc": array.array("L", []), "n_ocorrencias": array.array("L", [])})['id_doc'].append(int_id_doc)
      self.indice.setdefault(token, {"id_doc": array.array("L", []), "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)
    
    self.n_docs += 1
    self.tamanho_doc[int_id_doc] = len(tokens)

# Testa o índice invertido com os três primeiros documentos da coleção
iidx = IndiceInvertido()
#conteudo_doc_0 = json.loads(searcher.doc(0).raw())['contents']
#conteudo_doc_1 = json.loads(searcher.doc(1).raw())['contents']
#conteudo_doc_2 = json.loads(searcher.doc(2).raw())['contents']
conteudo_doc_0 = "teste " * 4
conteudo_doc_1 = "alskfj lasdk asf"
conteudo_doc_2 = "lasdk teste braco"
iidx.adiciona_doc("0", conteudo_doc_0)
iidx.adiciona_doc("1", conteudo_doc_1)
iidx.adiciona_doc("2", conteudo_doc_2)
print(iidx.tokenizar(conteudo_doc_2))
print(f'0: {conteudo_doc_0}\n1: {conteudo_doc_1}\n2: {conteudo_doc_2}\n{iidx.indice}\n{iidx.tamanho_doc}')

['lasdk', 'test', 'braco']
0: teste teste teste teste 
1: alskfj lasdk asf
2: lasdk teste braco
{'test': {'id_doc': array('L', [0, 2]), 'n_ocorrencias': array('L', [4, 1])}, 'alskfj': {'id_doc': array('L', [1]), 'n_ocorrencias': array('L', [1])}, 'lasdk': {'id_doc': array('L', [1, 2]), 'n_ocorrencias': array('L', [1, 1])}, 'asf': {'id_doc': array('L', [1]), 'n_ocorrencias': array('L', [1])}, 'braco': {'id_doc': array('L', [2]), 'n_ocorrencias': array('L', [1])}}
{0: 4, 1: 3, 2: 3}
CPU times: user 3.77 ms, sys: 728 µs, total: 4.49 ms
Wall time: 3.88 ms


Gera o índice e depois salva em um arquivo pickle.

In [ ]:
%%time
import pickle

iidx_ms_marco = IndiceInvertido()

def carregar_indice_invertido_em_iidx_ms_marco():
  if gerar_indice_implementado:
    # Demorando cerca de 25 min pra gerar o índice
    with open('collections/msmarco-passage/collection.tsv', encoding='utf-8') as f:
      for idx, line in enumerate(f):
        doc_id, contents = line.rstrip().split('\t')
        iidx_ms_marco.adiciona_doc(doc_id, contents)

        if idx % 250000 == 0:
          print(idx)

    print('Finalizou a indexação.')
    print('Gerando arquivo de índice:')
    with open('iidx_ms_marco.pickle', 'wb') as f:
      pickle.dump(iidx_ms_marco.indice, f)
    print('Gerando arquivo de tamanho do documento:')
    with open('tamanho_doc_ms_marco.pickle', 'wb') as f:
      pickle.dump(iidx_ms_marco.tamanho_doc, f)

    # Copia arquivos de índice e tamanho do documento para o google drive:
    print('Copiando arquivo de índice para Drive')
    !cp iidx_ms_marco.pickle '/content/drive/My Drive/IA368-DD_deep_learning_busca/'  # type: ignore
    print('Copiando arquivo de tamanho de documento para Drive')
    !cp tamanho_doc_ms_marco.pickle '/content/drive/My Drive/IA368-DD_deep_learning_busca/'   # type: ignore

  else:
    # Demorando cerca de 1min pra copiar as informações do drive e abrir aqui
    print('Copiando arquivo de índice do drive...')
    !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/iidx_ms_marco.pickle' './iidx_ms_marco.pickle'  # type: ignore
    print('Copiando arquivo de tamanho do documento do drive...')
    !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/tamanho_doc_ms_marco.pickle' './tamanho_doc_ms_marco.pickle'  # type: ignore

    # Remonta o índice invertido
    print('Lendo arquivo de índice...')
    with open('iidx_ms_marco.pickle', 'rb') as f:
      iidx_ms_marco.indice = pickle.load(f)
    # Remonta o índice de tamanho de documento
    print('Lendo arquivo de tamanho do documento...')
    with open('tamanho_doc_ms_marco.pickle', 'rb') as f:
      iidx_ms_marco.tamanho_doc = pickle.load(f)
    # Recalcula o total de documentos
    iidx_ms_marco.n_docs = len(iidx_ms_marco.tamanho_doc)

# Nos testes durante o desenvolvimento é útil, pois posso ir mudando o índice as vezes. Na versão final nem precisaria
carregar_indice_invertido_em_iidx_ms_marco()

Copiando arquivo de índice do drive...
Copiando arquivo de tamanho do documento do drive...
Lendo arquivo de índice...
Lendo arquivo de tamanho do documento...
CPU times: user 6.24 s, sys: 6.76 s, total: 13 s
Wall time: 40.7 s



Faz alguns testes rápidos com o índice gerado:

In [ ]:
print(f"Total de documentos com a palavra what: {len(iidx_ms_marco.indice['what']['id_doc'] )}")
print(f"Total de documentos com a palavra oil: {len(iidx_ms_marco.indice['oil']['id_doc'] )}")
print(f"Total de documentos no índice: {iidx_ms_marco.n_docs}")
print(f"Id do primeiro documento que tem a palavra what: {iidx_ms_marco.indice['what']['id_doc'][0]}")
print(f"Total de tokens do primeiro documento que tem a palavra what: {iidx_ms_marco.tamanho_doc[0]}")

Total de documentos com a palavra what: 543257
Total de documentos com a palavra oil: 106501
Total de documentos no índice: 8841823
Id do primeiro documento que tem a palavra what: 0
Total de tokens do primeiro documento que tem a palavra what: 30


O exercício pediu um "buscador booleano/bag of words". Pra mim não ficou claro se são buscadores distintos ou não. Uma pesquisa no Google e uma conversa com o ChatGPT parece indicar que são coisas iguais e cujo objetivo é retornar uma busca dos documentos que dão match com a query, sem o ranking.

Entretanto, é possível também definir ranking também para os documentos. O ChatGPT indicou, por exemplo, fazer o ranking usando TF-IDF ou contar os termos da query que aparecem na lista de documentos final.

Como um dos objetivos do exercício é a implementação de um buscador com TF-IDF, vamos deixar pra usá-lo posteriormente. Assim, para simplificar, vamos montar o ranking de acordo se o termo aparece no documento ou não.

Por exemplo, suponha que a query seja "numero primo" e existem dois documentos retornados assim:

1. meu primo gosta de numero primo
2. o primo dela viajou

No primeiro caso o score será 2, pois tanto a palavra primo quanto a palavra numero aparecem. No documento 2 o score será 1, pois apenas a palavra primo aparece no documento.

Note que nessa contagem de palavras há a possibilidade de um outro cálculo do score que será dado pela contagem de ocorrências da palavra. No exemplo acima, o score do documento 1 será 3 (2 para a palavra primo e 1 para a palavra numero) e o score do documento 2 será 1 (a palavra primo aparece apenas uma vez).

<br>

Para implemenar isso, vamos seguir o exemplo da classe do Lucene e criar uma classe PesquisaSimples que recebe o nosso índice invertido. E aí criamos um método pesquisar que recebe uma query e o tipo de cálculo do score. O tipo pode ser a string "1" ou a string "OCORRENCIA", ou seja, se for "1", cada termo da query contribuirá com 0 ou 1 para o cálculo do score. Se for "OCORRENCIA", contribuirá com a quantidade de vezes que ele aparece no documento.

<br>

Um outro detalhe aqui é o que o método pesquisar também tem um parâmetro chamado remover_stopwords_nltk. Esse parâmetro serve para reduzir ainda mais a query para desconsiderar palavras que são consideradas stopwords na bibliteca NLTK mas que não são na biblioteca do Lucene por padrão. Note que isso não altera o índice, apenas reduz a query. Isso não tem impacto na formulação da pesquisa booleana (mas não pode no caso do TF-IDF, por isso não usaremos essa redução da query lá).

In [ ]:
%%time

class PesquisaSimples:

  lista_stopwords_nltk = set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

  def __init__(self, indiceInvertido=IndiceInvertido()):
    self.indiceInvertido = indiceInvertido
    
  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  # O tipo de pesquisa pode ser '1' ou 'OCORRENCIA'
  def pesquisar(self, query, tipo='1', remover_stopwords_nltk=False):
    # Tokeniza a query
    tokens = self.tokenizar(query)
    if (remover_stopwords_nltk):
      tokens = set(tokens).difference(PesquisaSimples.lista_stopwords_nltk)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = {}

    # Na pesquisa booleana vou montar o ranking adicionando 1 se o termo está no documento, 0 caso contrário
    # No bag of words estou considerando quantas ocorrências ele aparece em cada documento.
    # Isso é feito através da função get_contribuicao_do_score_no_doc
    get_contribuicao_do_score_no_doc = (lambda score : 1) if tipo == '1' else (lambda score : score)

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:

      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # apenas ignora o termo (já que não tem como saber) e continua...
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que tem o token e a quantidade de ocorrências que o token
      # tem em cada documento:
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      n_ocorrencias_do_token_no_doc = self.indiceInvertido.indice[token]['n_ocorrencias']

      # Percorre essa lista conjuntamente. Aqui é que é gerado o score de cada documento
      # No caso da pesquisa booleana, vai adicionando 1 caso o documento apareça na query ou não
      # No caso de bag of words, vai adicionando o score. Pra não ter que ficar comparando
      # toda hora que tipo de pesquisa é e nem para duplicar código em dois métodos, 
      # estou usando a função get_contribuicao_do_score_no_doc definida antes do loop
      for id_doc, n_ocorrencias in zip(docs_que_tem_token, n_ocorrencias_do_token_no_doc):
        docs_retornado_com_score[id_doc] = docs_retornado_com_score.get(id_doc, 0) + get_contribuicao_do_score_no_doc(n_ocorrencias)

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)
  

# Testando com a primeira query:
query = topics[735922]["title"]
print(f'Query: {query}')
print(f'Tokens da query: {iidx_ms_marco.tokenizar(query)}')
pesquisa_simples = PesquisaSimples(iidx_ms_marco)
resultado = pesquisa_simples.pesquisar(query, "1", True)
print(resultado[0:10])

Query: what is crimp oil
Tokens da query: ['what', 'crimp', 'oil']
[(180247, 2), (180248, 2), (281697, 2), (281701, 2), (1240692, 2), (1432224, 2), (1798285, 2), (2342899, 2), (2381629, 2), (2523268, 2)]
CPU times: user 82.7 ms, sys: 4.08 ms, total: 86.8 ms
Wall time: 85.3 ms


Vamos ver os 10 primeiros resultados de uma query trazida pelo BM25 e pelo buscador booleano:

In [ ]:
%%time

query = topics[735922]['title']

print(f'Pesquisando a query: {query}')

# Primeiro pesquisa usando o BM25 do Lucene:
searcher = LuceneSearcher('indexes/lucene-index-msmarco-passage')
searcher.set_bm25(k1=float(0.9), b=float(0.4))
hits = searcher.search(query, k=20)

# Agora pesquisa usando o Buscador booleano
resultado = pesquisa_simples.pesquisar(query, "1", True)

print('\nResultado do LuceneSearcher (BM-25):\n')
for i in range(0, 10):
  jsondoc = json.loads(hits[i].raw)
  print(f'{i+1:2} {hits[i].score:.5f} {jsondoc["contents"][:160]}...')

print('\nResultado do PesquisaSimples:\n')
print(f'Documentos retornados: {resultado[0:10]}\n')
for i in range(0, 10):
  # PesquisaSimples indexa em int. O índice do sistema do Lucene, em string
  str_id_doc = str(resultado[i][0])
  score = resultado[i][1]
  conteudo = json.loads(searcher.doc(str_id_doc).raw())['contents']
  print(f'{score:.5f} {conteudo[:160]}...')

Pesquisando a query: what is crimp oil

Resultado do LuceneSearcher (BM-25):

 1 9.60990 Definition of crimped in the Definitions.net dictionary. Meaning of crimped. What does crimped mean? Information and translations of crimped in the most compreh...
 2 9.39730 Description: There has always been a debate as to what is the best form of crimping - hexagonal or indent. There is no straight answer as the crimping method de...
 3 9.39330 What does crimped mean? Definitions for crimped Here are all the possible meanings and translations of the word crimped....
 4 9.20850 Effect of Crimp Depth on Shotshells. Crimp depth of a finished shotshell reload is an important dimension to monitor for consistent ballistics and safe loads. T...
 5 8.95510 Crimp Oil is a 100% natural blend of essential oils and plant extracts that aids in the recovery of climbing-related injuries....
 6 8.66260 Directions. 1  Heat your oven to 225 degrees Fahrenheit. 2  On a large sheet of aluminum foil, shiny side up, 

Define um método para rodar todas as 200 queries de teste do TREC DL 2020:

In [ ]:
%%time

def run_all_queries_pesquisa_simples(file, topics, pesquisaSimples, tipo, remover_stopwords_nltk=False):
  with open(file, 'w') as runfile:
    cnt = 0
    print('Running {} queries in total'.format(len(topics)))
    for id in topics:
      query = topics[id]['title']
      hits = pesquisa_simples.pesquisar(query, tipo, remover_stopwords_nltk)
      for i in range(0, min(len(hits), 30)): # Pega os primeiros 30 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} '.format(id, hits[i][0], i+1, hits[i][1]) + "TIPO_" + tipo + '\n')
      cnt += 1
      if cnt % 100 == 0:
        print(f'{cnt} queries completed')

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 13.4 µs


In [ ]:
%%time

print('Rodando pesquisa simples considerando que o score é um por termo...')
run_all_queries_pesquisa_simples('run-pesquisa-simples_score_um_por_termo.dl20.txt', topics, pesquisa_simples, '1', True)
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run-pesquisa-simples_score_um_por_termo.dl20.txt

Rodando pesquisa simples considerando que o score é um por termo...
Running 200 queries in total
100 queries completed
200 queries completed
2023-03-08 09:54:21.033893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:54:21.034041: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:54:21.034068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned 

In [ ]:
%%time

print('Rodando pesquisa simples considerando que o score é o total de ocorrências dos termos da query...')
run_all_queries_pesquisa_simples('run-pesquisa-simples_score_ocorrencia_por_termo.dl20.txt', topics, pesquisa_simples, 'bow', True)
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run-pesquisa-simples_score_ocorrencia_por_termo.dl20.txt

Rodando pesquisa simples considerando que o score é o total de ocorrências dos termos da query...
Running 200 queries in total
100 queries completed
200 queries completed
2023-03-08 09:56:20.077229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:56:20.077398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 09:56:20.077427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure th

### IV. Buscador com TF-IDF e BM25

O buscador com TF-IDF foi implementado na classe PesquisaComTFIDF. Essa classe inicia igual a classe PesquisaSimples, recebendo um objeto do tipo IndiceInvertido (já com o índice calculado). Além disso, recebe os parâmetros k1 e b.

A partir daí, o construtor chama a função precalcula_idf para fazer o cálculo do IDF para todos os termos presentes no índice. Esse cálculo é rápido, então é bem tranquilo fazer isso no construtor.

Note que a estrutura do objeto IndiceInvertido.indice é: 

    {
      "token": {
        "id_doc": array.array contendo as ids dos documentos,
        "n_ocorrencias": array.array contendo quantas ocorrências o token aparece no documento equivalente
      }
    }

Podemos alterar essa estrutura para já computar o IDF e deixar ele no índice junto com as outras informações:

    {
      "token": {
        "id_doc": array.array contendo as ids dos documentos,
        "n_ocorrencias": array.array contendo quantas ocorrências o token aparece no documento equivalente
        "idf": int
        "tf_idf": dict contendo o score do tf_idf para o par token/documento
        "bm25": dict contendo o score calculado pelo BM25 para o par token/documento
      }
    }

Note acima que, além do idf calculado no construtor foi inserido também uma chave "tf_idf". Essa chave é o cálculo do tf_idf para todo par token/doc. No nosso caso não há necessidade de fazer isso no índice todo logo de uma vez ao inicializar o buscador. Isso deixaria a inicialização um pouco lenta. O que é feito é, ao se deparar com um termo de pesquisa pela primeira vez, calcula-se o tf_idf de todos os documentos que tem aquele termo e salva igual na estrutura acima. Na próxima vez que o termo de pesquisa aparece, o termo já está calculado e pode ser recuperado do cache. O efeito desse cache foi a redução do tempo no cálculo nas 200 queries de ~9 minutos para ~6:30 minutos.



In [ ]:
%%time

import math

class PesquisaComTFIDF:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.adicionar_ao_idf = adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
  
  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_tf_idf_para_um_token_e_salva(self, token):
    # O cálculo do tf_idf exibe apenas multiplicar o idf do termo (que já temos calculado) pelo tf(termo, documento)
    # A fórmula do tf = a frequência do termo no documento dividido pelo total de termos do documento
    # Logo, tf_idf = idf * frequencia_token_no_doc / tamanho_doc
    #             - o idf do token já está precalculado e consta do índice: self.indiceInvertido.indice[token]['idf']
    #             - o tamanho_doc também já consta do índice: self.indiceInvertido.tamanho_doc[id_doc] (para certo id_doc)
    #             - a frequência do token no documento também consta do índice: self.indiceInvertido.indice[token]['n_ocorrencias']
    # Juntando tudo:
    # Recupera o idf
    idf_token = self.indiceInvertido.indice[token]['idf']
    # Calcula o tf_idf
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    #tf_idf = { id_doc: idf_token * freq_token_no_doc / self.indiceInvertido.tamanho_doc[id_doc] for (id_doc, freq_token_no_doc) in zip_id_freq ]}
    tf_idf = array.array('f', [idf_token * freq_token_no_doc / self.indiceInvertido.tamanho_doc[id_doc] for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o tf_idf no índice
    self.indiceInvertido.indice[token]['tf_idf'] = tf_idf

  def calcula_bm25_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    # Para não ter que ficar fazendo várias multiplicações/divisões, vamos resolver o que dá pra resolver antes.
    # Sobre isso, o denominador fica: k1 - k1*b + k1*b*tamanho_doc/avgdl
    # Então precisamos de três variáveis:
    #         idf_x_k1_mais_1 = idf*(k1 + 1)
    #         k1_menos_k1_x_b = k1 - k1*b
    #         k1_x_b_d_avgdl = k1*b/avgdl
    idf_x_k1_mais_1 = self.indiceInvertido.indice[token]['idf']*(self.k1 + 1)
    k1_menos_k1_x_b = self.k1 - self.k1*self.b
    k1_x_b_d_avgdl = self.k1 * self.b / self.avgdl

    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    #bm25 = { id_doc: idf_x_k1_mais_1 * freq_token_no_doc / (freq_token_no_doc + k1_menos_k1_x_b + k1_x_b_d_avgdl*self.indiceInvertido.tamanho_doc[id_doc]) for (id_doc, freq_token_no_doc) in zip_id_freq }
    bm25 = array.array('f', [ idf_x_k1_mais_1 * freq_token_no_doc / (freq_token_no_doc + k1_menos_k1_x_b + k1_x_b_d_avgdl*self.indiceInvertido.tamanho_doc[id_doc]) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['bm25'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query, metodo = 'tf_idf'):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Função para o cálculo de score
    funcao_calc_salvar_scores_docs_deste_token = self.calcula_tf_idf_para_um_token_e_salva if metodo == 'tf_idf' else self.calcula_bm25_para_um_token_e_salva

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      
      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o tf_idf relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if metodo not in self.indiceInvertido.indice[token].keys():
        funcao_calc_salvar_scores_docs_deste_token(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token][metodo]

      # Agora já temos calculado o tf_idf de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 1.31 ms, sys: 0 ns, total: 1.31 ms
Wall time: 1.39 ms


Para testar a implementação do BM25, vamos refazer o exercício anterior. São 5 documentos e o resultado do score esperado para o BM25 (considerando adicionar_ao_idf = 1) é:

5. primo gosta numero primo. 0,953423382
1. numero primo distante. 0,848369848
4. primo bagunceiro. 0,636667008
2. numero natural grande. 0,295230582
3. grande numero dourado escorregadio. 0,260989921

Para o TF-IDF é:

5. primo gosta numero primo. 0,3415
1. numero primo distante. 0,27566
4. primo bagunceiro. 0,2695
2. numero natural grande. 0,096
3. grande numero dourado escorregadio. 0,072

E o IDF com adicionar +1:

numero: 0,287
primo: 0,539

Testando o código:

In [ ]:
iidx_numero_primo = IndiceInvertido()
iidx_numero_primo.adiciona_doc(1, "numero primo distante")
iidx_numero_primo.adiciona_doc(2, "numero natural grande")
iidx_numero_primo.adiciona_doc(3, "grande numero dourado escorregadio")
iidx_numero_primo.adiciona_doc(4, "primo bagunceiro")
iidx_numero_primo.adiciona_doc(5, "primo gosta numero primo")

pesquisa_com_tf_idf = PesquisaComTFIDF(iidx_numero_primo, k1 = 1.2, b = 0.75, adicionar_ao_idf = 1)
resultado = pesquisa_com_tf_idf.pesquisar("numero primo", 'bm25')

print('Pesquisa com BM25:\n')
for (id, score) in resultado:
  print(id, score)

resultado = pesquisa_com_tf_idf.pesquisar("numero primo", 'tf_idf')
print('\n\nPesquisa com TF-IDF:\n')
for (id, score) in resultado:
  print(id, score)

Pesquisa com BM25:

5 0.953423410654068
1 0.848369836807251
4 0.636667013168335
2 0.29523056745529175
3 0.2609899342060089


Pesquisa com TF-IDF:

5 0.3414187803864479
1 0.27555952966213226
4 0.2694982588291168
2 0.09589402377605438
3 0.07192052155733109


Pra testar, vamos usar a mesma query anterior e comparar com o BM-25 do Pyserini com mesmos parâmetros:

In [ ]:
%%time
query = topics[735922]['title']

print(f'Pesquisando a query no BM25 do Lucene: {query}')

# Primeiro pesquisa usando o BM25 do Lucene:
searcher = LuceneSearcher('indexes/lucene-index-msmarco-passage')
searcher.set_bm25(k1=float(0.9), b=float(0.4))
hits = searcher.search(query, k=20)

# Agora pesquisa usando o Buscador com TFIDF
pesquisa_com_tf_idf = PesquisaComTFIDF(iidx_ms_marco, 1)
resultado = pesquisa_com_tf_idf.pesquisar(query, 'bm25')

print('\nResultado do LuceneSearcher (BM-25):\n')
for i in range(0, 10):
  jsondoc = json.loads(hits[i].raw)
  print(f'{i+1:2} {hits[i].score:.5f} {jsondoc["contents"][:160]}...')

print('\nResultado da pesquisa implementada:\n')
print(f'Documentos retornados: {resultado[0:10]}')
for i in range(0, 10):
  # PesquisaSimples indexa em int. O índice do sistema do Lucene, em string
  str_id_doc = str(resultado[i][0])
  score = resultado[i][1]
  conteudo = json.loads(searcher.doc(str_id_doc).raw())['contents']
  print(f'{score:.5f} {conteudo[:160]}...')

Pesquisando a query no BM25 do Lucene: what is crimp oil

Resultado do LuceneSearcher (BM-25):

 1 9.60990 Definition of crimped in the Definitions.net dictionary. Meaning of crimped. What does crimped mean? Information and translations of crimped in the most compreh...
 2 9.39730 Description: There has always been a debate as to what is the best form of crimping - hexagonal or indent. There is no straight answer as the crimping method de...
 3 9.39330 What does crimped mean? Definitions for crimped Here are all the possible meanings and translations of the word crimped....
 4 9.20850 Effect of Crimp Depth on Shotshells. Crimp depth of a finished shotshell reload is an important dimension to monitor for consistent ballistics and safe loads. T...
 5 8.95510 Crimp Oil is a 100% natural blend of essential oils and plant extracts that aids in the recovery of climbing-related injuries....
 6 8.66260 Directions. 1  Heat your oven to 225 degrees Fahrenheit. 2  On a large sheet of aluminum foi

E agora rodar na base de TREC DL 20 e ver o resultado:

In [ ]:
%%time
def run_all_queries_pesquisa_com_tf_idf(file, topics, pesquisa_com_tf_idf, metodo = 'tf_idf'):
  with open(file, 'w') as runfile:
    cnt = 0
    print('Running {} queries in total'.format(len(topics)))
    for id in topics:
      query = topics[id]['title']
      hits = pesquisa_com_tf_idf.pesquisar(query, metodo)
      for i in range(0, min(len(hits), 30)): # Pega os primeiros 30 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} PESQUISA_'.format(id, hits[i][0], i+1, hits[i][1]) + metodo + '\n')
      cnt += 1
      if cnt % 10 == 0:
        print(f'{cnt} queries completed')


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [ ]:
%%time

print('Rodando pesquisa com TF-IDF...')
run_all_queries_pesquisa_com_tf_idf('run-pesquisa-com-tf-idf.dl20.txt', topics, pesquisa_com_tf_idf)
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run-pesquisa-com-tf-idf.dl20.txt

Rodando pesquisa com TF-IDF...
Running 200 queries in total
10 queries completed
20 queries completed
30 queries completed
40 queries completed
50 queries completed
60 queries completed
70 queries completed
80 queries completed
90 queries completed
100 queries completed
110 queries completed
120 queries completed
130 queries completed
140 queries completed
150 queries completed
160 queries completed
170 queries completed
180 queries completed
190 queries completed
200 queries completed
2023-03-08 10:03:10.681800: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 10:03:10.681964: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open

In [ ]:
%%time
print('Rodando pesquisa com BM25...')
run_all_queries_pesquisa_com_tf_idf('run-pesquisa-com-bm25.dl20.txt', topics, pesquisa_com_tf_idf, 'bm25')
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run-pesquisa-com-bm25.dl20.txt

Rodando pesquisa com BM25...
Running 200 queries in total
10 queries completed
20 queries completed
30 queries completed
40 queries completed
50 queries completed
60 queries completed
70 queries completed
80 queries completed
90 queries completed
100 queries completed
110 queries completed
120 queries completed
130 queries completed
140 queries completed
150 queries completed
160 queries completed
170 queries completed
180 queries completed
190 queries completed
200 queries completed
2023-03-08 10:10:06.105814: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 10:10:06.105967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open s

### V. Conclusões

<br>

#### V.A Resultados consolidados

Neste exercício a base TREC DL-2020 foi avaliada com o Pyserini usando 5 buscadores:

1. O primeiro usou o BM25 do Lucene com parâmetros (k1, b) = (0.9, 0.4). Esses não são os valores que maximizam os resultados para a base, mas foram escolhidos para replicar o trabalho de [Ma et al](https://castorini.github.io/pyserini/2cr/msmarco-v1-passage.html).

2. O segundo buscador foi o buscador booleano implementado que adiciona ao cálculo do score 1 ou 0 dependendo se o termo está presente no documento ou não.

3. O terceiro buscador analisado foi o mesmo que o anterior, mas configurado para adicionar n ao cálculo do score, onde n é a quantidade de vezes que a palavra pesquisa aparece no documento.

4. O quarto foi a implementação do TF-IDF. Nesse caso, utilizei o IDF somando + 1 ao operando do logaritmo para usar a mesma formulação que o Lucene usa (esse é um parâmetro configurável na implementação).

5. O último foi um buscador BM25 com a formulação padrão com (k1, b) = (0.9, 0.4).

Os resultados obtidos para cada buscador considerando a métrica nDCG@10 e o tempo de execução para executar 200 queries e avaliar o resultado foram (o tempo foi coletado em uma rodada sequencial deste caderno e deve ser visto apenas para comparação entre eles numa comparação mais grosseira) :

<center>

| Buscador              | nDCG@10 | Tempo de execução |
|-----------------------|---------|-------------------|
| Lucene BM25           | 0.4796  | 6.47s             |
| Buscador booleano (1) | 0.3427  | 1min 26s          |
| Buscador booleano (n) | 0.0509  | 1min 39s          |
| Buscador com TF-IDF   | 0.1574  | 6min 15s          |
| Buscador BM25         | 0.4823  | 6min 9s           |


</center>

<br>


A primeira observação relevante é que o buscador booleano que leva em consideração o total de ocorrências da palavra no documento tem um resultado muito ruim. Isso possivelmente ocorre porque documentos grandes vão ter os termos da query aparecendo várias vezes, fazendo com que documentos grandes sejam sempre privilegiados no resultado simplesmente por terem palavras demais. Ou seja, considerar unicamente a frequência absoluta de termos é algo que traz danos a busca.

Uma outra questão é que o buscador booleano usou mais stopwords que o TF-IDF e BM25. Isso teve um impacto muito grande no booleano. Enquanto o nDCG@10 estava na casa dos 0.19 enquanto usava apenas o pré-processamento do Lucene, ao considerar também as stopwords do NLTK houve melhora para ~0.34.

Em relação ao buscador do BM25  implementado, eu esperava que o resultado obtido fosse exatamente o mesmo. Entretanto, houve diferença no resultado. Apesar de terem ordenação parecidas, o score que eu obtive foi bem diferente do calculado no Lucene.

Ainda sobre o TF-IDF, uma questão interessante que surgiu aqui é que o resultado de ~0.15 poderia ser aumentado para essa coleção de documentos se, em vez de IDF, usássemos IDF^2.

<br>

#### V.B Dificuldades encontradas e como foram solucionadas

Geração do índice invertido:

- Na criação do índice invertido, usei uma estrutura de dict para associar a cada token uma lista de documentos em que aquele token era encontrado. A lista era um tipo list() e, os elementos, string. Esse processo ocupou muita memória, mas estava funcionando.

- O segundo passo na criação do índice foi inserir também a informação de quantas vezes o token aparece naquele documento. A tentativa inicial foi usar list() de int(), mas houve estouro de memória (ambiente com 25 GB).

- O estouro da memória foi resolvido após alterar a lista de ids de string para int. Como cada id de documento é numérico, não houve problemas nisso. Assim foi possível gerar essas informações em memória. Entretanto a memória estourou após tentar gravar o índice em arquivo.

- Após diversas alternativas foi possível corrigir isso alterando o tipo de dados de list() para array.array(). Houve redução significativa da memória ocupada (mais de 50%), o que possibilitou salvar o índice em arquivo e posteriormente carregá-lo.

Lentidão no buscador do TF-IDF:

- Na primeira implementação funcional do buscador, como o cálculo do IDF é rápido, o construtor do buscador já calculava o IDF para todos os termos e armazenava. Entretanto, o cálculo do TF para cada termo da query era feito sempre que uma query era apresentada. O resultado final foi que a busca nas 200 queries estava levando cerca de 9 minutos para finalizar.

- Como muitas palavras (tokens) são usadas em várias queries (e essas palavras ainda costumam aparecer em muitos documentos), havia um desperdício recalculando o TF de um par query/documento sempre que um termo era apresentado. Isso foi resolvido criando no índice um cache para o cálculo do TF. Quando um termo é apresentado ao buscador pela primeira vez ele faz o cálculo e armazena no índice. Quando já tem, pega direto do índice. Isso reduziu o tempo para ~ 6:30 minutos.

- Após a retirada do loop de algumas somas e multiplicações do cálculo do TF, o tempo reduziu para menos de 6 minutos, ou seja, em média menos de 2 segundos por query.

<br>

#### V.C Dúvidas

1. Como tratar termos que aparecem mais de uma vez na query? Na implementação feita, se um termo aparece duas vezes na query o score dele é contado duas vezes. Quais os impactos disso?

<br>

#### V.D Ideias de colegas

Usei as seguintes ideias dos colegas:

- Guardar o índice no Google Drive: conversa de Marcus Borela e Júlia Tessler.

- Uso do wget para download do script que faz a conversão de tsv para json: notebook do Gustavo Bartz

- Uso de stopwords do NLTK para diminuir a query (usada na pesquisa booleana): conversa com Monique Monteiro